<a href="https://colab.research.google.com/github/LasseS123/Emotional-Claim-Generation-Verification/blob/main/test_robustness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data generation

In [ ]:
# Schritt A: Notwendige Bibliotheken installieren (mit Update für bitsandbytes)
!pip install --upgrade transformers torch bitsandbytes accelerate pandas tqdm

# Schritt B: Alle nötigen Module importieren
import os
import time
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from google.colab import userdata
from typing import Optional
from tqdm.notebook import tqdm



# 1. Hugging Face Token sicher laden
try:
    hf_token = userdata.get('HF_Token')
    print("✅ Hugging Face Token erfolgreich geladen.")
except Exception as e:
    print("🔴 FEHLER: Hast du den HF_TOKEN im Secret Manager gespeichert?")

# 2. Modell-ID für Qwen 7B festlegen
MODEL_ID = "Qwen/Qwen1.5-7B-Chat"

# 3. Konfiguration für 4-Bit-Quantisierung, um Speicher zu sparen
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 4. Tokenizer und Modell laden (Dieser Schritt dauert bei 7B weniger lange als bei 72B!)
print(f"Lade Tokenizer für {MODEL_ID}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=hf_token)

print(f"Lade Modell '{MODEL_ID}'...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto", # Weist das Modell automatisch der A100-GPU zu
    quantization_config=quantization_config,
    token=hf_token
)
print(f"✅ {MODEL_ID}-Modell erfolgreich geladen und bereit!")

# 5. Pipeline für die Textgenerierung erstellen
qwen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=250,
    model_kwargs={"temperature": 0.3}
)
print("✅ Text-Generierungs-Pipeline ist bereit.")

# Anweisungen für die 6 Emotionen (bleiben gleich)
EMOTION_INSTRUCTIONS = {
    "anger": "Rewrite the sentence to express strong anger and frustration. Preserve all numbers and names. Do not change the factual meaning.",
    "disgust": "Rewrite the sentence to express strong disgust and revulsion. Preserve all numbers and names. Do not change the factual meaning.",
    "fear": "Rewrite the sentence to express strong fear, anxiety, and worry. Preserve all numbers and names. Do not change the factual meaning.",
    "happiness": "Rewrite the sentence to express strong happiness, joy, and excitement. Preserve all numbers and names. Do not change the factual meaning.",
    "sadness": "Rewrite the sentence to express strong sadness, disappointment, and grief. Preserve all numbers and names. Do not change the factual meaning.",
    "surprise": "Rewrite the sentence to express strong surprise and astonishment. Preserve all numbers and names. Do not change the factual meaning."
}

# =========================
# Kernfunktionen (angepasst für Qwen)
# =========================

def qwen_colab_transform(base_text: str, instruction: str) -> str:
    """Sendet eine Anfrage an das lokal geladene Qwen-Modell."""
    # Spezielles Chat-Format für Qwen
    messages = [
        {"role": "system", "content": "You are a rewriting assistant. Preserve the factual meaning exactly. Output only the rewritten sentence."},
        {"role": "user", "content": f"{instruction}\n\nClaim:\n{base_text}"},
    ]
    prompt = qwen_pipeline.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    try:
        outputs = qwen_pipeline(
            prompt,
            do_sample=True,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id # Unterdrückt eine Warnung
        )

        # Extrahiere nur den generierten Text
        generated_text = outputs[0]['generated_text'][len(prompt):].strip()
        return generated_text
    except Exception as e:
        print(f"Fehler bei der Inferenz: {e}")
        return base_text # Fallback auf Originaltext

# =========================
# Hauptskript
# =========================

def main():
    # Passe diesen Dateinamen an, falls deine Input-Datei anders heißt
    input_filename = "sample100_gold_pairs.csv"

    try:
        df = pd.read_csv(input_filename)
        print(f"\n✅ Eingabedatei '{input_filename}' erfolgreich geladen.")
    except FileNotFoundError:
        print(f"🔴 FEHLER: Die Datei '{input_filename}' wurde nicht gefunden. Hast du sie hochgeladen?")
        return

    rows = []
    for i, row in tqdm(df.iterrows(), total=len(df), desc="Generiere emotionale Varianten"):
        base = str(row["query_text"]).strip()
        qid = row["query_id"]
        doc = row["doc_id"]

        output_row = {"query_id": qid, "original_query_text": base, "doc_id": doc}

        for emotion, instruction in EMOTION_INSTRUCTIONS.items():
            # Verwende die neue, für Qwen angepasste Funktion
            rewritten_text = qwen_colab_transform(base, instruction)
            output_row[f"{emotion}_version"] = rewritten_text

        rows.append(output_row)

    out = pd.DataFrame(rows)
    # Definiere die Spaltenreihenfolge für eine saubere Ausgabe
    column_order = [
        "query_id", "original_query_text", "doc_id",
        "anger_version", "disgust_version", "fear_version",
        "happiness_version", "sadness_version", "surprise_version"
    ]
    out = out[column_order]

    # Ändere den Ausgabedateinamen, um das Modell widerzuspiegeln
    output_filename = "emotional_dataset_6_emotions_Qwen7B.csv"
    out.to_csv(output_filename, index=False, encoding="utf-8")
    print(f"\n✅ Fertig! Die Datei '{output_filename}' wurde erfolgreich erstellt und kann heruntergeladen werden.")

# Skript ausführen
if __name__ == "__main__":
    main()

# Robustheits-Analyse für 6 Emotionen

Dieses Notebook testet, wie robust ein Embedding-Modell (`all-MiniLM-L6-v2`) gegenüber 6 verschiedenen emotionalen Varianten von Suchanfragen ist. Es vergleicht die Suchergebnisse der Originalanfrage mit denen der emotionalen Varianten und berechnet Metriken wie Recall@20, MRR@20 und die durchschnittliche Rangveränderung.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 1. Installation der notwendigen Bibliotheken
!pip install sentence-transformers pandas numpy tqdm

## 2. Konfiguration & Daten laden

In diesem Schritt werden alle erforderlichen Daten und das KI-Modell geladen. Bitte stelle sicher, dass die folgenden Dateien in die Colab-Umgebung hochgeladen wurden:
- `vclaims_corpus_min.csv`
- `emotional_dataset_6_emotions.csv`

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
from tqdm.notebook import tqdm

# Lade das Korpus, gegen das gesucht wird
try:
    corpus_df = pd.read_csv("vclaims_corpus_min.csv")
    corpus_df = corpus_df.rename(columns={"vclaim_id": "doc_id", "vclaim_text": "doc_text"})
    corpus_docs = dict(zip(corpus_df.doc_id, corpus_df.doc_text))
    print(f"✅ Korpus mit {len(corpus_docs)} Dokumenten geladen.")
except FileNotFoundError:
    print("🔴 FEHLER: vclaims_corpus_min.csv nicht gefunden. Bitte lade die Datei hoch.")

# Lade das Dataset mit den 6 Emotionen
try:
    eval_df = pd.read_csv("emotional_dataset_6_emotions_Qwen7B.csv")
    print(f"✅ Emotionales Dataset mit {len(eval_df)} Zeilen geladen.")
except FileNotFoundError:
    print("🔴 FEHLER: emotional_dataset_6_emotions.csv nicht gefunden. Bitte lade die Datei hoch.")


# Definiere die zu testenden Emotions-Spalten
EMOTION_COLUMNS = [
    "anger_version",
    "disgust_version",
    "fear_version",
    "happiness_version",
    "sadness_version",
    "surprise_version"
]

# Definiere das Embedding-Modell
MODEL_NAME = 'all-MiniLM-L6-v2'
model = SentenceTransformer(MODEL_NAME)
print(f"\n✅ Sentence-Transformer-Modell '{MODEL_NAME}' geladen.")

## 3. Korpus-Embeddings vorberechnen

Um die Auswertung zu beschleunigen, werden die Embeddings für alle Dokumente im Korpus einmalig im Voraus berechnet und im GPU-Speicher gehalten.

In [ ]:
print("Erstelle Embeddings für das Korpus... (kann einen Moment dauern)")
corpus_ids = list(corpus_docs.keys())
corpus_embeddings = model.encode(
    [corpus_docs[doc_id] for doc_id in corpus_ids],
    convert_to_tensor=True,
    show_progress_bar=True
)
print("✅ Korpus-Embeddings sind fertig.")

## 4. Haupt-Auswertung

Jetzt wird die eigentliche Analyse durchgeführt. Das Skript iteriert durch jede Zeile deines Datasets, erstellt Embeddings für die Originalanfrage und für jede der sechs emotionalen Varianten und vergleicht die Suchergebnisse.

In [ ]:
def calculate_recall(retrieved_ids, relevant_id, k=20):
    return 1 if relevant_id in retrieved_ids[:k] else 0

def calculate_mrr(retrieved_ids, relevant_id, k=20):
    try:
        rank = retrieved_ids[:k].index(relevant_id) + 1
        return 1.0 / rank
    except ValueError:
        return 0.0

def calculate_rank_change(base_rank, style_rank):
    if base_rank is None or style_rank is None:
        return None
    return style_rank - base_rank # Positiv = Rang wurde schlechter

def get_rank(retrieved_ids, relevant_id):
    try:
        return retrieved_ids.index(relevant_id) + 1
    except ValueError:
        return None

results = []

# tqdm sorgt für einen schönen Fortschrittsbalken
for _, row in tqdm(eval_df.iterrows(), total=len(eval_df), desc="Werte Abfragen aus"):
    query_id = row['query_id']
    doc_id_gold = row['doc_id']

    # --- Embedding für die Original-Abfrage ---
    query_base_text = row['original_query_text']
    query_base_emb = model.encode(query_base_text, convert_to_tensor=True)
    hits_base = util.semantic_search(query_base_emb, corpus_embeddings, top_k=100)[0]
    retrieved_base = [corpus_ids[hit['corpus_id']] for hit in hits_base]
    rank_base = get_rank(retrieved_base, doc_id_gold)

    # --- Iteriere durch jede Emotions-Spalte ---
    for emotion_style in EMOTION_COLUMNS:
        query_style_text = row[emotion_style]

        # Embedding für die emotionale Variante
        query_style_emb = model.encode(query_style_text, convert_to_tensor=True)
        hits_style = util.semantic_search(query_style_emb, corpus_embeddings, top_k=100)[0]
        retrieved_style = [corpus_ids[hit['corpus_id']] for hit in hits_style]
        rank_style = get_rank(retrieved_style, doc_id_gold)

        # Speichere alle Ergebnisse
        results.append({
            'query_id': query_id,
            'style': emotion_style.replace("_version", ""), # Mache aus "anger_version" -> "anger"
            'doc_id_gold': doc_id_gold,
            'Recall@20_BASE': calculate_recall(retrieved_base, doc_id_gold),
            'Recall@20_STYLE': calculate_recall(retrieved_style, doc_id_gold),
            'MRR@20_BASE': calculate_mrr(retrieved_base, doc_id_gold),
            'MRR@20_STYLE': calculate_mrr(retrieved_style, doc_id_gold),
            'Rank_BASE': rank_base,
            'Rank_STYLE': rank_style,
            'Rank_Change': calculate_rank_change(rank_base, rank_style),
        })

results_df = pd.DataFrame(results)
print("\n✅ Auswertung abgeschlossen.")

## 5. Ergebnisse zusammenfassen und anzeigen

Die finalen Ergebnisse werden aggregiert und in einer übersichtlichen Tabelle dargestellt. Außerdem werden die detaillierten und die zusammengefassten Ergebnisse als CSV-Dateien gespeichert.

In [ ]:
# Berechne die durchschnittlichen Metriken pro Emotion
summary_list = []

for style in results_df['style'].unique():
    style_df = results_df[results_df['style'] == style]

    # Filtere nur die Paare, bei denen das Originaldokument gefunden wurde
    paired_df = style_df.dropna(subset=['Rank_BASE', 'Rank_STYLE'])

    summary_list.append({
        'style': style,
        'N_total': len(style_df),
        'N_paired': len(paired_df),
        'Recall@20_BASE': style_df['Recall@20_BASE'].mean(),
        'Recall@20_STYLE': style_df['Recall@20_STYLE'].mean(),
        'MRR@20_BASE': style_df['MRR@20_BASE'].mean(),
        'MRR@20_STYLE': style_df['MRR@20_STYLE'].mean(),
        'Avg_Rank_Change': paired_df['Rank_Change'].mean(),
        'Std_Rank_Change': paired_df['Rank_Change'].std(),
    })

summary_df = pd.DataFrame(summary_list).sort_values('style').reset_index(drop=True)

# Berechne die prozentuale Veränderung
summary_df['ΔRecall(%)'] = (summary_df['Recall@20_STYLE'] - summary_df['Recall@20_BASE']) / summary_df['Recall@20_BASE'] * 100
summary_df['ΔMRR(%)'] = (summary_df['MRR@20_STYLE'] - summary_df['MRR@20_BASE']) / summary_df['MRR@20_BASE'] * 100

# Formatiere die Ausgabe für bessere Lesbarkeit
pd.options.display.float_format = '{:,.3f}'.format
summary_to_display = summary_df[[
    'style', 'N_total', 'N_paired',
    'Recall@20_STYLE', 'ΔRecall(%)',
    'MRR@20_STYLE', 'ΔMRR(%)',
    'Avg_Rank_Change'
]]

print("--- Zusammenfassung der Robustheits-Analyse ---")
display(summary_to_display)

# Speichere die detaillierten und zusammengefassten Ergebnisse
results_df.to_csv("eval_results_6_emotions_detailed.csv", index=False)
summary_df.to_csv("eval_summary_6_emotions.csv", index=False)
print("\nGespeichert: 'eval_results_6_emotions_detailed.csv' und 'eval_summary_6_emotions.csv'")